# Scene characterization

In this present notebook we play around with some functions from the mitsuba library to get geometric information of a scene

In [ ]:
import os
import math

if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
tf.get_logger().setLevel('ERROR')

# Set random seed for reproducibility
sionna.config.seed = 42

from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera, watt_to_dbm
from sionna.mimo.precoding import normalize_precoding_power, grid_of_beams_dft
from sionna.utils import log10

# Python importings
import gc
from datetime import date
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import ConvexHull

try:
    import alphashape
except:
    os.system("pip install alphashape")

try:
    import descartes
except:
    os.system("pip install descartes")

try:
    import pandas as pd
except:
    os.system("pip install pandas")
    import pandas as pd

# Required alpha shape libraries
try:
    from matplotlib.collections import LineCollection
    from shapely.ops import unary_union, polygonize
    import shapely.geometry as geometry
    from scipy.spatial import Delaunay
    from shapely.geometry import Point
    import pylab as pl
    from descartes import PolygonPatch

except:
    os.system("pip install shapely")
    os.system("pip install scipy")
    os.system("pip install pylab")
    os.system("pip install descartes")

    from matplotlib.collections import LineCollection
    from shapely.ops import unary_union, polygonize
    import shapely.geometry as geometry
    from scipy.spatial import Delaunay
    from shapely.geometry import Point
    import pylab as pl
    from descartes import PolygonPatch
    

## Paths

In [ ]:
# paths
datasets = ['sionna_madrid', 'sionna_madrid_concrete_ground']
scene_path = 'sionna_madrid/2800001/scene.xml'

# scene = sionna.rt.load_scene(scene_path)
# print(f"This is the size of the scene: {scene.size}")
# scene_size = np.array(scene.size)
# for i in scene_size:
#     print(i)

# csv name
csv_name = 'scene_characterization.csv'

## Functions

In [ ]:
# in this cell we define all the utility funtions

def compute_face_area(v1, v2, v3):
    # Compute the area of a triangle using the cross product
    # v1, v2, v3 are the 3 vertices of the triangle
    # Area = 1/2 * |(v2 - v1) x (v3 - v1)| (cross product magnitude)
    edge1 = v2 - v1
    edge2 = v3 - v1
    cross_product = np.cross(edge1, edge2)
    return 0.5 * np.linalg.norm(cross_product)

def compute_building_area(shape):
    # Retrieve the vertices and faces of the mesh
    vertices = shape.vertex_position()
    faces = shape.faces()
    
    total_area = 0.0
    for face in faces:
        # Each face is a triangle defined by 3 vertices
        v1 = vertices[face[0]]
        v2 = vertices[face[1]]
        v3 = vertices[face[2]]
        total_area += compute_face_area(v1, v2, v3)
    
    return total_area

def computing_area(min_coords, max_coords):
    """
    Compute the area of the buildings with respect to the groundplane in 2D
    """
    min_x, min_y, min_z = min_coords
    max_x, max_y, max_z = max_coords

    length = max_x - min_x
    width = max_y - min_y

    return length * width

def building_volume(min_coords, max_coords):
    """
    Calculate the volume of a building given its min and max coordinates.

    :param min_coords: A tuple (min_x, min_y, min_z)
    :param max_coords: A tuple (max_x, max_y, max_z)
    :return: The volume of the building
    """
    min_x, min_y, min_z = min_coords
    max_x, max_y, max_z = max_coords
    
    # Calculate the dimensions
    length = max_x - min_x
    width = max_y - min_y
    height = max_z - min_z

    return length * width * height

def check_length_dict(dictionary):
    for key, value in dictionary.items():
        try:
            print(f"The length of the value for key '{key}' is: {len(value)}")
        except TypeError:
            print(f"Value for key '{key}' is not a type that supports len().")

# new added functions
def clip_coordinates(coords, min_limit=-200, max_limit=200):
    """
    Clips the (x, y) coordinates to ensure they stay within the defined scene bounds.
    """
    coords[:, 0] = np.clip(coords[:, 0], min_limit, max_limit)  # Clip x values
    coords[:, 1] = np.clip(coords[:, 1], min_limit, max_limit)  # Clip y values
    return coords

def compute_area_of_building(vertices):
    """
    Compute the area covered by a building based on its vertices' (x, y) coordinates.
    The building's area is computed by finding the convex hull of the vertices.
    """

    test = False
    # Only consider (x, y) coordinates, ignoring z
    coords = np.array(vertices)[:, :2]  # Extract only x, y coordinates

    # Clip coordinates to ensure they stay within the bounds of the scene
    coords = clip_coordinates(coords)
    # print(f"\nThese are the coordinates that we are passing to the convex hull: {coords}\n")

    # Compute the convex hull of the (x, y) points
    try:
        hull = ConvexHull(coords)
        area = hull.volume  # For 2D convex hull, 'volume' corresponds to area

        # now let us test the computation
        if test:
            # plot the blue dots and the black line
            plt.plot(coords[:,0], coords[:,1], 'o')
            for simplex in hull.simplices:
                plt.plot(coords[simplex, 0], coords[simplex, 1], 'k-')

            # plot the red envelope
            plt.plot(coords[hull.vertices,0], coords[hull.vertices,1], 'r--', lw=2)
            plt.plot(coords[hull.vertices[0],0], coords[hull.vertices[0],1], 'ro')
            plt.show()
        return area, coords

    except:
        # If the convex hull cannot be computed (for example, if all points are collinear),
        # return 0 area
        return 0


def reconstructing_scene(buildings, scene_dir):
    """
    This function takes as input all vertices from each shape and reconstructs
    the original scenario but subjected to both x and y limits
    """

    # Plotting
    plt.figure(figsize=(8, 8))
    plt.xlim(-200, 200)
    plt.ylim(-200, 200)

    print(f"These are the shapes: {buildings}")
    print(f"Number of shapes: {len(buildings)}")
    # Iterate through each building and plot the convex hull or outline
    for building in buildings:
        # For a convex hull (to represent the building outline)
        building = np.array(building)
        print(f"This is the type of a building: {type(building)}")
        print(f"This is the length of a building: {len(building)}")
        print(f"This is a building: {building}")
        if len(building) != 0:
            hull = ConvexHull(building)
         
            # Plot the building outline using the convex hull
            plt.plot(building[hull.vertices, 0], building[hull.vertices, 1], 'r-', lw=2)  # Convex hull outline
            plt.fill(building[hull.vertices, 0], building[hull.vertices, 1], 'r', alpha=0.3)  # Fill the area

        else:
            continue

    # Plot individual vertices for each building (optional)
    for building in buildings:
        building = np.array(building)
        if len(building) != 0:
            plt.scatter(building[:, 0], building[:, 1], label="Building Vertices")
        else:
            continue

    # Add labels and show the plot
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.title('Building Silhouettes on the Ground Plane')
    plt.grid(True)
    # plt.legend(loc='upper left')
    plt.gca().set_aspect('equal', adjustable='box')  # Make sure the aspect ratio is equal

    # Save the plot to a PDF file
    plt.savefig(f"{scene_dir}/scene_reconstruction.pdf", format='pdf')

    # show the figure
    plt.show()

def reconstructing_scene_alpha(buildings, scene_dir, alpha):
    """
    This function takes as input all vertices from each shape and reconstructs
    the original scenario but subjected to both x and y limits. Uses Alpha Shapes 
    for concave hulls instead of Convex Hulls to handle irregular shapes.
    """

    # Plotting
    plt.figure(figsize=(8, 8))
    # plt.xlim(-200, 200)
    # plt.ylim(-200, 200)

    # Iterate through each building and plot the alpha shape (concave hull)
    for building in buildings:
        building = np.array(building)

        # Compute the alpha shape (concave hull) for the building
        alpha_shape = alphashape.alphashape(building, alpha=alpha)

        # Check if the alpha shape is a Polygon or MultiPolygon
        if alpha_shape.geom_type == 'Polygon':
            # If it's a single polygon, plot it
            plt.plot(*alpha_shape.exterior.xy, 'r-', lw=2)  # Alpha shape outline
            plt.fill(*alpha_shape.exterior.xy, 'r', alpha=0.3)  # Fill the area
    
        elif alpha_shape.geom_type == 'MultiPolygon':
            # If it's a MultiPolygon, iterate over each polygon inside it
            for poly in alpha_shape.geoms:
                plt.plot(*poly.exterior.xy, 'r-', lw=2)  # Alpha shape outline
                plt.fill(*poly.exterior.xy, 'r', alpha=1)  # Fill the area

    # Plot individual vertices for each building (optional)
    for building in buildings:
        building = np.array(building)
        plt.scatter(building[:, 0], building[:, 1], label="Building Vertices")

    # Add labels and show the plot
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.title('Building Silhouettes on the Ground Plane')
    plt.grid(True)
    plt.gca().set_aspect('equal', adjustable='box')  # Ensure equal aspect ratio

    # Save the plot to a PDF file
    plt.savefig(f"{scene_dir}/scene_reconstruction.pdf", format='pdf')

    # Show the figure
    plt.show()


### Alpha shape implementation FUNCTIONS

In [ ]:

def plot_polygon(polygon):
    fig = pl.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    margin = .3

    x_min, y_min, x_max, y_max = polygon.bounds

    ax.set_xlim([x_min-margin, x_max+margin])
    ax.set_ylim([y_min-margin, y_max+margin])
    patch = PolygonPatch(polygon, fc='#999999', ec='#000000', fill=True, zorder=-1)
    ax.add_patch(patch)
    return fig

def alpha_shape(points, alpha):
    """
    Compute the alpha shape (concave hull) of a set of points.

    @param points: Iterable container of points.
    @param alpha: alpha value to influence the gooeyness of the border. Smaller
                  numbers don't fall inward as much as larger numbers. Too large,
                  and you lose everything!
    """
    if len(points) < 4:
        # When you have a triangle, there is no sense in computing an alpha
        # shape.
        return geometry.MultiPoint(list(points)).convex_hull

    def add_edge(edges, edge_points, coords, i, j):
        """Add a line between the i-th and j-th points, if not in the list already"""
        if (i, j) in edges or (j, i) in edges:
            # already added
            return
        edges.add((i, j))
        edge_points.append(coords[[i, j]])

    coords = np.array([point.coords[0] for point in points])

    tri = Delaunay(coords)
    edges = set()
    edge_points = []
    # loop over triangles:
    # ia, ib, ic = indices of corner points of the triangle
    for ia, ib, ic in tri.simplices:
        pa = coords[ia]
        pb = coords[ib]
        pc = coords[ic]

        # Lengths of sides of triangle
        a = math.sqrt((pa[0]-pb[0])**2 + (pa[1]-pb[1])**2)
        b = math.sqrt((pb[0]-pc[0])**2 + (pb[1]-pc[1])**2)
        c = math.sqrt((pc[0]-pa[0])**2 + (pc[1]-pa[1])**2)

        # Semiperimeter of triangle
        s = (a + b + c)/2.0

        # Area of triangle by Heron's formula
        area = math.sqrt(s*(s-a)*(s-b)*(s-c))
        circum_r = a*b*c/(4.0*area)

        # Here's the radius filter.
        #print circum_r
        if circum_r < 1.0/alpha:
            add_edge(edges, edge_points, coords, ia, ib)
            add_edge(edges, edge_points, coords, ib, ic)
            add_edge(edges, edge_points, coords, ic, ia)

    m = geometry.MultiLineString(edge_points)
    triangles = list(polygonize(m))
    return unary_union(triangles), edge_points # before it implemented cascaded_union


def delaunay_triangulation(buildings, scene_dir):
    '''
    This function implements the code from cell: In [18] from the following link:
    https://gist.github.com/dwyerk/10561690
    - buildings variable contains all the building coordinates divided by lists, 
    in order to differentiate them
    - scene_dir contains the str of the given path
    '''

    # let us firstly transform our coordinates to an understandable format
    new_points = []
    for building in buildings:
        new_points.append([Point(coord[0], coord[1]) for coord in building])

    print(f"\nThis is the new_points BEFORE flatten: {new_points}")
    print(f"This is the length of new_points BEFORE flatten: {len(new_points)}")
    print(f"This is the type of new_points BEFORE flatten: {type(new_points)}\n")

    # if needed Flatten the list of Point objects into a single list of points (if necessary)

    # new_points = [point for sublist in new_points for point in sublist]

    # print(f"\nThis is the new_points AFTER flatten: {new_points}")
    # print(f"This is the length of new_points AFTER flatten: {len(new_points)}")
    # print(f"This is the type of new_points AFTER flatten: {type(new_points)}\n")

    alpha = 0.001
    for building in new_points:
        concave_hull, edge_points = alpha_shape(building, alpha=alpha)

        #print concave_hull
        lines = LineCollection(edge_points)
        pl.figure(figsize=(10,10))
        pl.title('Alpha={0} Delaunay triangulation'.format(alpha))
        pl.gca().add_collection(lines)

        # new_points = [geometry.shape(point['geometry']) for point in new_shapefile] # geometry is a library & new_shape_file = fiona.open(input_shapefile) & input_shapefile = "efwegwg.shp"
        delaunay_points = np.array([point.coords[0] for point in building]) # point.coords[0] basically takes the frist position of a list of tuples: [(1.0, 2.0)] -> (1.0, 2.0)
        plt.plot(delaunay_points[:, 0], delaunay_points[:, 1], 'o', color='#f16824')

        # input_shapefile = 'demo_poly_scaled_points_join.shp'
        # new_shapefile = fiona.open(input_shapefile)
        # new_points = [geometry.shape(point['geometry']) for point in new_shapefile]
        # print "We have {0} points!".format(len(new_points))
        x = [p.coords.xy[0] for p in building]
        y = [p.coords.xy[1] for p in building]

        # en realidad lo que tenemos que hacer es no convertir nada de las coordendaas, y coger directamente las que tenemos y seleccionar solo la coordenada x
        _ = plot_polygon(concave_hull)
        _ = plt.plot(x,y,'o', color='#f16824')



## Test code

In [ ]:
verbose = False

datasets = ['sionna_madrid', 'sionna_madrid_concrete_ground']
# datasets = ['sionna_madrid_concrete_ground']
for directory in datasets:
    attributes = {"scene_name": [],
                  "scenes_size": [],
                  "num_elements": [],
                  "max_vertex_count": [],
                  "min_vertex_count": [],
                  "avg_vertex_count": [],
                  "tallest_building": [],
                  "shortest_building": [],
                  "avg_building_height": [],
                  "tot_num_faces": [],
                  "avg_num_faces_building": [],
                  "max_num_faces_building": [],
                  # "min_coords_biggest_building": [],
                  # "max_coords_biggest_building": [],
                  # "scene_total_area": [],
                  "tot_buildings_area": [], # respecto al ground_plane (solo eje x y eje y)
                  "avg_building_area": [],
                  "max_building_area": [],
                  "min_occuppied_percentage_area": [],
                  "max_occuppied_percentage_area": [],
                  "avg_occuppied_percentage_area": [],
                  "total_occuppied_percentage_area": []
                  # "tot_buildings_volume_max": [], # esto es maximo porque puede que el edificio no sea geométrico y por lo tanto siempre va a estar por dentro
                  # "avg_building_volume": [],
                  # "max_building_volume": []
                  # "num_streets": [],
                  # "total_street_area": [],
                  # "biggest_street": [],
                 }

    if verbose: print(f'Working on: {directory}')
    for name in os.listdir(directory):
        if verbose: print(f"Directory: {name}")

        if os.path.isdir(os.path.join(directory, name)) and name != ".ipynb_checkpoints":
            scene_dir = os.path.join(directory, name)
            print(f"We are in the following directory: {name}")
            xml_file = [os.path.join(scene_dir, file) for file in os.listdir(scene_dir) if file.endswith(".xml")]
            if verbose: print(f"XML file -> {xml_file}")

            # loading the scene
            scene = sionna.rt.load_scene(xml_file[0])
            scene_size = np.array(scene.size)
            scene_size = scene_size.tolist()
            scene_area = scene_size[0]*scene_size[1]
            if verbose: print(f"This is the size of the scene: {scene_size}")
            if verbose: print(f"This is the area of the scene: {scene_area}")

            # since for our measurements, we are not considering building parts, ourside the groundplane
            # we have to limit the area size to [400, 400, max_building_height]
            scene_size[0] = 400
            scene_size[1] = 400

            #similarly for the area of the scene
            scene_area = float(scene_size[0]*scene_size[1])

            # let us get the Mitsuba scene
            mi_scene = scene.mi_scene
            mi_shapes = mi_scene.shapes()
            # print(f"These are all the shapes: {mi_shapes}")

            num_scene_elements = len(mi_shapes)
            if verbose: print(f"This is the number of scene elements (also the ground): {num_scene_elements}")

            if num_scene_elements < 2:
                continue # if no building are found, then, dont even mind putting it inside the csv, because we'd have to delete it from there too

            else:
                # let us get now get all the coordinates
                building_heights = []
                # building_volumes = []
                building_areas = []
                max_coords = []
                min_coords = []
                num_faces = []
                vertex_counts = []
                occuppied_percentage_area_per_building = []
                all_shape_vertices = [] # for the whole scene reconstruction
                sum_vertices_coords = 0

                for shape in mi_shapes:
                    element_id = shape.id()
                    if verbose: print(f"This is the shape: {shape}")
                    if verbose: print(f"This is the id of the element: {element_id}")

                    if element_id != 'mesh-ground': #do not get the ground
                        coords = shape.bbox()
                        min_coord = coords.min
                        min_coords.append(min_coord) # append it to the list
                        max_coord = coords.max
                        max_coords.append(max_coord) # append it to the list

                        # Output min and max coordinates of the building
                        if verbose: print(f"Min coordinate: {min_coord}")
                        if verbose: print(f"Max coordinate: {max_coord}")
                        # max_coord_list = list(max_coords)
                
                        # let's get the height of the building:
                        height = max_coord[len(max_coord)-1] # always get the max coordinate of the object, if not height = 0
                        building_heights.append(height)
                        if verbose: print(f"Height of the building: {height}")

                        # let us compute the area with the new method
                        # area = computing_area(min_coord, max_coord)
                        vertices_coords = []
                        vertex_count = shape.vertex_count()
                        vertex_counts.append(vertex_count)
                        if verbose: print(f"This is the shape's vertex count: {vertex_counts}")
                        for vertex in range(vertex_count):
                            if verbose: print(f"This is the vertex_count: {vertex}")
                            vertex_positions = shape.vertex_position(index=vertex)
                            if verbose: print(f"These are vertex_positions: {vertex_positions}")
                            if vertex_positions[2] == 0:
                                if verbose: print(f"These are the vertex positions that have z = 0: {vertex_positions}")
                                vertex_positions = np.array(vertex_positions) # transform into np array to manipulate the tensor
                                if verbose: print(f"These are the vertex_positions: {vertex_positions}")
                                vertices_coords.append(vertex_positions[0][:2]) # we discard the z coordinate
                            if verbose: print(f"Associated vertex position: {vertex_positions}")

                        # let us add all the vertices coordinates, to reconstruct the whole scenario
                        all_shape_vertices.append(vertices_coords)
                        # ahora solo nos tenemos que quedar con los vértices que tengan z = 0

                        # let's compute the new area with the convex hull
                        if verbose: print(f"These are the vertices_coords: {vertices_coords}")
                        # if not vertices_coords: continue
                        if vertices_coords:
                            building_area, coords = compute_area_of_building(vertices_coords)
                            sum_vertices_coords += len(vertices_coords)
                            building_areas.append(building_area)
                            if verbose: print(f"Covered Area of the Building on Ground: {building_area:.2f} square units")

                            # let us now get the occupied percentage area
                            building_occuppied_area_percentage = 100*building_area/scene_area
                            if verbose: print(f"This is the occuppied percentage area: {building_occuppied_area_percentage}")
                            occuppied_percentage_area_per_building.append(building_occuppied_area_percentage)

                        # let's compute the volume of the building
                        # volume = building_volume(min_coord, max_coord)
                        # building_volumes.append(volume)
                        # if verbose: print(f"Volume of the building: {volume}")

                        # let's get the faces
                        face_count = shape.face_count()
                        num_faces.append(face_count)
                        if verbose: print(f"This is the shape's face count: {face_count}")
                
                    else:
                        continue

                # para tener todos los vértices en un solo lugar, tenemos que ponerlo por shape, no podemos poner todos de una y mezclados
                # reconstructing_scene_alpha(all_shape_vertices, scene_dir, alpha=20)
                print(f"These are all the shape vertices: {all_shape_vertices}")
                reconstructing_scene(all_shape_vertices, scene_dir)

                # let us get the coordinates of the building with the gratest volume
                # greatest_volume = max(building_volumes)
                # max_coords_biggest_vol = max_coords[building_volumes.index(greatest_volume)]
                # min_coords_biggest_vol = min_coords[building_volumes.index(greatest_volume)]

                greatest_area = max(building_areas)
    
                # let us now append all  information
                # let us now append all  information
                attributes['scene_name'].append(name)
                attributes['scenes_size'].append(scene_size)
                attributes['num_elements'].append(num_scene_elements)
                attributes['max_vertex_count'].append(max(vertex_counts))
                attributes['min_vertex_count'].append(min(vertex_counts))
                attributes['avg_vertex_count'].append(np.average(vertex_counts))
                attributes['tallest_building'].append(max(np.array(building_heights)))
                attributes['shortest_building'].append(min(building_heights))
                attributes['avg_building_height'].append(np.average(building_heights))
                attributes['tot_num_faces'].append(sum(num_faces))
                attributes['avg_num_faces_building'].append(np.average(num_faces))
                attributes['max_num_faces_building'].append(max(num_faces))
                attributes['tot_buildings_area'].append(sum(building_areas))
                attributes['avg_building_area'].append(np.average(building_areas))
                attributes['max_building_area'].append(greatest_area)
                # attributes['scene_total_area'].append(scene_area)
                attributes['min_occuppied_percentage_area'].append(min(occuppied_percentage_area_per_building))
                attributes['max_occuppied_percentage_area'].append(max(occuppied_percentage_area_per_building))
                attributes['avg_occuppied_percentage_area'].append(np.average(occuppied_percentage_area_per_building))
                attributes['total_occuppied_percentage_area'].append(sum(occuppied_percentage_area_per_building))
                # attributes['tot_buildings_volume_max'].append(sum(building_volumes))
                # attributes['avg_building_volume'].append(np.average(building_volumes))
                # attributes['max_building_volume'].append(greatest_volume)
                # attributes['max_coords_biggest_building'].append(np.array(max_coords_biggest_vol)) # according to the volume
                # attributes['min_coords_biggest_building'].append(np.array(min_coords_biggest_vol))
                if verbose: print(f"This is the final dictionary: {attributes}")

    # start saving results (this is key when simulating long batches)
    sim_df = pd.DataFrame.from_dict(attributes)
    # sim_df.set_index(attributes['scene_name'], inplace=True)

    # check if the file exists
    csv_name = str(f"scene_charact_{directory}" + str(date.today()) + ".csv")
    csv_path = os.path.join(directory, csv_name)
    if Path(csv_path).is_file():
        # sim_csv = sim_df.to_csv(csv_path, index=False) # drop ID column in the CSV
        sim_df.to_csv(csv_path, mode = 'a', index=False, header=False) # drop ID column in the CSV and
                                                                       # header = False to avoid repetition of labels
    else:
        sim_csv = sim_df.to_csv(csv_path, index=False) # drop ID column in the CSV
        # print('He entrado donde tengo que entrar')


In [ ]:
tot_vol = building_volume([-33.746910095214844, 36.94416809082031, 0.0],[57.545921325683594, 88.74097442626953, 15.829158782958984])
print(f"This is the total volume: {tot_vol}")

In [ ]:
# print(f"This is the dicionary: {attributes}")

# now let us compute the length of each attribute one by one
for keys in attributes.keys():
    print(f"{keys} has {len(attributes[keys])} rows")

## Let us check for individual scenes

In [ ]:
verbose = False
path = 'sionna_madrid/2801503/scene.xml'
# 2810061
# 
attributes = {# "scene_name": [],
                  "scenes_size": [],
                  "num_elements": [],
                  "max_vertex_count": [],
                  "min_vertex_count": [],
                  "avg_vertex_count": [],
                  "tallest_building": [],
                  "shortest_building": [],
                  "avg_building_height": [],
                  "tot_num_faces": [],
                  "avg_num_faces_building": [],
                  "max_num_faces_building": [],
                  # "min_coords_biggest_building": [],
                  # "max_coords_biggest_building": [],
                  "scene_total_area": [],
                  "tot_buildings_area": [], # respecto al ground_plane (solo eje x y eje y)
                  "avg_building_area": [],
                  "max_building_area": [],
                  "min_occuppied_percentage_area": [],
                  "max_occuppied_percentage_area": [],
                  "avg_occuppied_percentage_area": [],
                  "total_occuppied_percentage_area": []
                  # "tot_buildings_volume_max": [], # esto es maximo porque puede que el edificio no sea geométrico y por lo tanto siempre va a estar por dentro
                  # "avg_building_volume": [],
                  # "max_building_volume": []
                  # "num_streets": [],
                  # "total_street_area": [],
                  # "biggest_street": [],
                 }

# loading the scene
scene = sionna.rt.load_scene(path)
scene_size = np.array(scene.size)
scene_size = scene_size.tolist()
scene_area = scene_size[0]*scene_size[1]
if verbose: print(f"This is the size of the scene: {scene_size}")
if verbose: print(f"This is the area of the scene: {scene_area}")

# since for our measurements, we are not considering building parts, ourside the groundplane
# we have to limit the area size to [400, 400, max_building_height]
scene_size[0] = 400
scene_size[1] = 400

#similarly for the area of the scene
scene_area = scene_size[0]*scene_size[1]

# let us get the Mitsuba scene
mi_scene = scene.mi_scene
mi_shapes = mi_scene.shapes()
# print(f"These are all the shapes: {mi_shapes}")

num_scene_elements = len(mi_shapes)
if verbose: print(f"This is the number of scene elements (also the ground): {num_scene_elements}")

# let us get now get all the coordinates
building_heights = []
# building_volumes = []
building_areas = []
max_coords = []
min_coords = []
num_faces = []
vertex_counts = []
occuppied_percentage_area_per_building = []

sum_vertices_coords = 0
all_shape_vertices = []

print(f"There are {len(mi_shapes)} shapes in the scene")

for shape in mi_shapes:
    element_id = shape.id()
    print(f"This is the shape: {shape}")
    if verbose: print(f"This is the id of the element: {element_id}")

    if element_id != 'mesh-ground': #do not get the ground
        coords = shape.bbox()
        min_coord = coords.min
        min_coords.append(min_coord) # append it to the list
        max_coord = coords.max
        max_coords.append(max_coord) # append it to the list

        # Output min and max coordinates of the building
        if verbose: print(f"Min coordinate: {min_coord}")
        if verbose: print(f"Max coordinate: {max_coord}")
        # max_coord_list = list(max_coords)

        # let's get the height of the building:
        height = max_coord[len(max_coord)-1] # always get the max coordinate of the object, if not height = 0
        building_heights.append(height)
        if verbose: print(f"Height of the building: {height}")

        # let us compute the area with the new method
        # area = computing_area(min_coord, max_coord)
        vertices_coords = []
        vertex_count = shape.vertex_count()
        vertex_counts.append(vertex_count)
        if verbose: print(f"This is the shape's vertex count: {vertex_counts}")
        for vertex in range(vertex_count):
            if verbose: print(f"This is the vertex_count: {vertex}")
            vertex_positions = shape.vertex_position(index=vertex)
            if vertex_positions[2] == 0:
                if verbose: print(f"These are the vertex positions that have z = 0: {vertex_positions}")
                vertex_positions = np.array(vertex_positions) # transform into np array to manipulate the tensor
                if verbose: print(f"These are the vertex_positions: {vertex_positions}")
                vertices_coords.append(vertex_positions[0][:2]) # we discard the z coordinate

        # let us add all the vertices coordinates, to reconstruct the whole scenario
        print(f"\nThese are the vertices_coords we are adding: {vertices_coords}\n")
        all_shape_vertices.append(vertices_coords)

        # ahora solo nos tenemos que quedar con los vértices que tengan z = 0


        # let's compute the new area with the convex hull
        if verbose: print(f"These are the vertices_coords: {vertices_coords}")
        # if not vertices_coords: continue
        if vertices_coords:
            print(f"This is the number of vertex coordiantes: {len(vertices_coords)}")
            building_area, coords = compute_area_of_building(vertices_coords)
            sum_vertices_coords += len(vertices_coords)
            building_areas.append(building_area)
            if verbose: print(f"Covered Area of the Building on Ground: {building_area:.2f} square units")

            # let us now get the occupied percentage area
            building_occuppied_area_percentage = 100*building_area/scene_area
            if verbose: print(f"This is the occuppied percentage area: {building_occuppied_area_percentage}")
            occuppied_percentage_area_per_building.append(building_occuppied_area_percentage)
 
        # let's compute the volume of the building
        # volume = building_volume(min_coord, max_coord)
        # building_volumes.append(volume)
        # if verbose: print(f"Volume of the building: {volume}")
        # let's get the faces
        face_count = shape.face_count()
        num_faces.append(face_count)
        if verbose: print(f"This is the shape's face count: {face_count}")

    else:
        continue

print(f"Types: {type(all_shape_vertices[0][0])}, actual element: {all_shape_vertices[0][0]}")
print(f"These are all the building vertices: {all_shape_vertices}")
if sum_vertices_coords == len(all_shape_vertices): print(f"Both terms match!!")
 
# para tener todos los vértices en un solo lugar, tenemos que ponerlo por shape, no podemos poner todos de una y mezclados
scene_dir = os.path.dirname(path)
# reconstructing_scene_alpha(all_shape_vertices, scene_dir)
# reconstructing_scene_alpha(all_shape_vertices, path, alpha=0.004)
# reconstructing_scene(all_shape_vertices, scene_dir)
delaunay_triangulation(all_shape_vertices, scene_dir)

# let us get the coordinates of the building with the gratest volume
# greatest_volume = max(building_volumes)
# max_coords_biggest_vol = max_coords[building_volumes.index(greatest_volume)]
# min_coords_biggest_vol = min_coords[building_volumes.index(greatest_volume)]

greatest_area = max(building_areas)

# let us now append all  information
# attributes['scene_name'].append(name)
attributes['scenes_size'].append(scene_size)
attributes['num_elements'].append(num_scene_elements)
attributes['max_vertex_count'].append(max(vertex_counts))
attributes['min_vertex_count'].append(min(vertex_counts))
attributes['avg_vertex_count'].append(np.average(vertex_counts))
attributes['tallest_building'].append(max(np.array(building_heights)))
attributes['shortest_building'].append(min(building_heights))
attributes['avg_building_height'].append(np.average(building_heights))
attributes['tot_num_faces'].append(sum(num_faces))
attributes['avg_num_faces_building'].append(np.average(num_faces))
attributes['max_num_faces_building'].append(max(num_faces))
attributes['tot_buildings_area'].append(sum(building_areas))
attributes['avg_building_area'].append(np.average(building_areas))
attributes['max_building_area'].append(greatest_area)
attributes['scene_total_area'].append(scene_area)
attributes['min_occuppied_percentage_area'].append(min(occuppied_percentage_area_per_building))
attributes['max_occuppied_percentage_area'].append(max(occuppied_percentage_area_per_building))
attributes['avg_occuppied_percentage_area'].append(np.average(occuppied_percentage_area_per_building))
attributes['total_occuppied_percentage_area'].append(sum(occuppied_percentage_area_per_building))
# attributes['tot_buildings_volume_max'].append(sum(building_volumes))
# attributes['avg_building_volume'].append(np.average(building_volumes))
# attributes['max_building_volume'].append(greatest_volume)
# attributes['max_coords_biggest_building'].append(np.array(max_coords_biggest_vol)) # according to the volume
# attributes['min_coords_biggest_building'].append(np.array(min_coords_biggest_vol))
if verbose: print(f"This is the final dictionary: {attributes}")

In [ ]:
# scene = load_scene(sionna.rt.scene.munich) # Merge shapes to speed-up computations
scene = sionna.rt.load_scene('sionna_madrid/2803683/scene.xml')
print(f"This is the size of the scene: {scene.size}")

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="cross")

# Create transmitter
tx = Transmitter(name="tx",
                 position=[8.5,21,27])

# Add transmitter instance to scene
scene.add(tx)

# Create a receiver
rx = Receiver(name="rx",
              position=[45,90,1.5])

# Add receiver instance to scene
scene.add(rx)

tx.look_at(rx) # Transmitter points towards receiver

scene.preview()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
import math

# Function to compute the area of a triangle given 3 vertices (ignoring z)
def triangle_area(v1, v2, v3):
    # Calculate the lengths of the sides of the triangle
    a = np.linalg.norm(v2 - v1)
    b = np.linalg.norm(v3 - v2)
    c = np.linalg.norm(v1 - v3)

    # Compute the semi-perimeter
    s = (a + b + c) / 2.0

    # Use Heron's formula to compute the area of the triangle
    area = math.sqrt(s * (s - a) * (s - b) * (s - c))
    return area

# Function to compute the area of a building's footprint (ignoring z-coordinate)
def compute_building_area(building_vertices):
    """
    Computes the total area of the building's footprint using Delaunay triangulation.
    Only considers the xy-coordinates, ignoring z.
    """
    # Convert building vertices to numpy array and ignore the z-coordinate
    building_vertices = np.array(building_vertices)[:, :2]

    # Perform Delaunay triangulation to break the shape into triangles
    delaunay = Delaunay(building_vertices)

    total_area = 0.0

    # Loop over each triangle in the Delaunay triangulation
    for simplex in delaunay.simplices:
        v1, v2, v3 = building_vertices[simplex[0]], building_vertices[simplex[1]], building_vertices[simplex[2]]
        
        # Compute the area of the triangle and sum it
        total_area += triangle_area(v1, v2, v3)

    return total_area, delaunay  # Return both the area and the Delaunay triangulation for plotting

# Example points for a building (list of (x, y) coordinates)
building_vertices = [
    [0.16, 3.98],
    [-0.48, 3.33],
    [-0.48, 4.53],
    [0.1, 3.67],
    [0.04, 5.67],
    [-7.94, 3.02],
    [-18.16, 3.07],
    [-0.15, 5.67],
    [-0.26, 5.14],
    [-0.1, 5.11],
    [-0.96, 5.48],
    [-0.03, 3.86],
    [-0.12, 3.16],
    [0.32, 4.64],
    [-0.1, 4.32],
    [-0.84, 4.28],
    [-0.56, 3.16],
    [-6.85, 3.28],
    [-0.7, 3.24],
    [-7.2, 3.03]
]

# Compute the total area of the building's footprint
building_area, delaunay = compute_building_area(building_vertices)
print(f"Total area of the building's footprint on the XY plane: {building_area:.2f} square units")

# Plotting the building footprint
plt.figure(figsize=(8, 8))

# Plot the vertices of the building
building_vertices = np.array(building_vertices)
plt.scatter(building_vertices[:, 0], building_vertices[:, 1], color='blue', label="Building Vertices")

# Plot the Delaunay triangulation
for simplex in delaunay.simplices:
    # Plot each triangle (use the vertex indices from the simplices)
    triangle = building_vertices[simplex]
    plt.fill(triangle[:, 0], triangle[:, 1], 'r', alpha=0.3)  # Fill the area of each triangle
    plt.plot(triangle[:, 0], triangle[:, 1], 'k-', lw=1)  # Outline of the triangle

plt.title("Building Footprint with Delaunay Triangulation")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.grid(True)
plt.gca().set_aspect('equal', adjustable='box')
plt.legend(loc='upper left')

# Show the plot
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import alphashape
from shapely.geometry import Polygon, MultiPolygon

# Example points for the building (list of (x, y) coordinates)
# building_vertices = [
#     [-10, 10],
#     [10, 10],
#     [10, -10],
#     [-10, -10]
# ]

# # Example interior points (patio inside the building)
# patio_vertices = [
#     [-5, 5],
#     [5, 5],  # This point is a duplicate; let's correct it
#     [5, -5],
#     [-5, -5]
# ]

# Combine the building and patio vertices into a single list
# all_vertices = building_vertices + patio_vertices
all_vertices = [
    [0.16, 3.98],
    [-0.48, 3.33],
    [-0.48, 4.53],
    [0.1, 3.67],
    [0.04, 5.67],
    [-7.94, 3.02],
    [-18.16, 3.07],
    [-0.15, 5.67],
    [-0.26, 5.14],
    [-0.1, 5.11],
    [-0.96, 5.48],
    [-0.03, 3.86],
    [-0.12, 3.16],
    [0.32, 4.64],
    [-0.1, 4.32],
    [-0.84, 4.28],
    [-0.56, 3.16],
    [-6.85, 3.28],
    [-0.7, 3.24],
    [-7.2, 3.03]
]

# Convert the list of points to a numpy array
all_points = np.array(all_vertices)

# Calculate the alpha shape (concave hull) using alpha = 0.2
alpha = 0
alpha_shape = alphashape.alphashape(all_points, alpha)

# Plotting the alpha shape (building footprint including the patio)
fig, ax = plt.subplots(figsize=(8, 8))

# Check if the alpha shape is a MultiPolygon or a single Polygon
if isinstance(alpha_shape, MultiPolygon):
    # If it's a MultiPolygon, iterate through each polygon and plot it
    for poly in alpha_shape.geoms:
        x, y = poly.exterior.xy
        ax.fill(x, y, color='red', alpha=0.3, label="Building Footprint (Alpha Shape)")
        ax.plot(x, y, color='black', lw=2)  # Outline of the building
else:
    # If it's a single polygon, plot it
    print(f'This is the alphashape: {alpha_shape}')
    x, y = alpha_shape.exterior.xy
    ax.fill(x, y, color='red', alpha=0.3, label="Building Footprint (Alpha Shape)")
    ax.plot(x, y, color='black', lw=2)  # Outline of the building

# Plot the vertices of the building and the patio
# building_points = np.array(building_vertices)
# patio_points = np.array(patio_vertices)
all_vertices = np.array(all_vertices)

ax.scatter(all_vertices[:, 0], all_vertices[:, 1], color='blue', label="Building Vertices")
# ax.scatter(patio_points[:, 0], patio_points[:, 1], color='green', label="Patio Vertices")

# Set plot limits
ax.set_xlim(-20, 20)
ax.set_ylim(-20, 20)
ax.set_title("Building Footprint with Alpha Shape (Including Patio)")
ax.set_xlabel("X Coordinate")
ax.set_ylabel("Y Coordinate")
ax.legend()

plt.show()

# Area of the alpha shape
area = alpha_shape.area
print(f"Area of the building footprint (including patio): {area:.2f} square units")

In [ ]:
from matplotlib.collections import LineCollection

for i in range(9):
    alpha = (i+1)*.1
    concave_hull, edge_points = alpha_shape(new_points, alpha=alpha)

    #print concave_hull
    lines = LineCollection(edge_points)
    pl.figure(figsize=(10,10))
    pl.title('Alpha={0} Delaunay triangulation'.format(alpha))
    pl.gca().add_collection(lines)

    # new_points = [geometry.shape(point['geometry']) for point in new_shapefile] # geometry is a library & new_shape_file = fiona.open(input_shapefile) & input_shapefile = "efwegwg.shp"
    delaunay_points = np.array([point.coords[0] for point in new_points]) # point.coords[0] basically takes the frist position of a list of tuples: [(1.0, 2.0)] -> (1.0, 2.0)
    pl.plot(delaunay_points[:,0], delaunay_points[:,1], 'o', hold=1, color='#f16824')

    _ = plot_polygon(concave_hull)
    _ = pl.plot(x,y,'o', color='#f16824')

In [ ]:
print(400*400)